# Agent-based Transportation Model

In [ ]:
import logging
import ssl; ssl._create_default_https_context = ssl._create_stdlib_context

from astropy import units
from pyrosm.data import sources
import ipywidgets as widgets
from IPython.display import display

from dpd.modeling import (
    TransportationModel,
    OriginDestinationDataFrame,
    Zones,
    ABTMMap,
)
from dpd.osm import OSMMap
from dpd.osrm import OSRM

logging.basicConfig(level=logging.INFO)

/Users/dbawssec/Library/Python/3.9/lib/python/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
region = widgets.Select(
    options=list(
        map(
            lambda x: x.replace("_", " ").title().replace("Of", "of"),
            sources.available["subregions"]["usa"],
        )
    ),
    description="Region",
    rows=10,
    value="District of Columbia"
)
display(region)

In [ ]:
osmmap = OSMMap(region.value)

In [ ]:
osmmap.plot_folium(filter_box=[-76.96, 39.02, -77.04, 38.96])

This is what was done in peopleModel.py

In [ ]:
import us
YEAR = "2017"

zones = Zones.from_uscensus(str(us.states.lookup(region.value).fips), YEAR)
od = OriginDestinationDataFrame(
    OriginDestinationDataFrame.from_lodes(
        us.states.lookup(region.value).abbr.lower(), YEAR
    ).sample(100)
)

This is what was done in routeModel.py

In [ ]:
osrm = OSRM(
    region="north-america/us/"
    + us.states.lookup(region.value).name.lower().replace(" ", "-"),
    profile_directory="/usr/local/Cellar/osrm-backend/5.24.0/share/osrm/profiles/",
    profile="car",
)
p = osrm.routed()
from time import sleep

sleep(1)
od.add_geometry_from_zones(zones)
od.add_route_hw_from_osrm(url_base="http://localhost:5000", mode="driving")
p.kill()

In [ ]:
model = TransportationModel()
abtmosmmap = ABTMMap(model, osmmap)
abtmosmmap.create_people_from_od(od.head(10))

This is what was done in simulation.py

In [ ]:
trajectories = abtmosmmap.simulate()

This is what was done in view.py

In [ ]:
#trajectories.add_speed()
trajectories.hvplot(line_width=10)

In [ ]:
import random

def random_color():
    return "#" + hex(random.randint(0, 0xFF))[2:] + hex(random.randint(0, 0xFF))[2:] + hex(random.randint(0, 0xFF))[2:]

features = []
for trajectory in trajectories.trajectories:
    color = random_color()
    df = trajectory.df.copy()
    df["previous_geometry"] = df["geometry"].shift()
    df["time"] = df.index
    df["previous_time"] = df["time"].shift()
    for _, row in df.iloc[1:].iterrows():
        coordinates = [
            [
                row["previous_geometry"].xy[0][0],
                row["previous_geometry"].xy[1][0]
            ],
            [
                row["geometry"].xy[0][0],
                row["geometry"].xy[1][0]
            ]
        ]
        times = [row["previous_time"].isoformat(), row["time"].isoformat()]
        features.append(
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": coordinates,
                },
                "properties": {
                    "times": times,
                    "style": {
                        "color": color,
                        "weight": 5,
                    },
                },
            }
        )

import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[38.913340600000055, -77.009092], zoom_start=16)

TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1S",
    add_last_point=True,
    transition_time=1000
).add_to(m)

m